In [8]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gower

In [9]:
# Data processing/preparation
headers = pd.read_csv("./data/tracks.csv", nrows = 3, header = None, sep=",")
combined_headers = headers.apply(lambda x: '.'.join(x.dropna().astype(str)), axis=0)
df = pd.read_csv("./data/tracks.csv", skiprows = 3, header = None, sep=",")
df.columns = combined_headers
print(df)

X = df.iloc[: ,1:]
Y = df.iloc[:, :1]

X_train,X_test,y_test,y_train = train_test_split(X,Y,test_size= 0.2)

        track_id  album.comments   album.date_created  album.date_released  \
0              2               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
1              3               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
2              5               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
3             10               0  2008-11-26 01:45:08  2008-02-06 00:00:00   
4             20               0  2008-11-26 01:45:05  2009-01-06 00:00:00   
...          ...             ...                  ...                  ...   
106569    155316               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106570    155317               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106571    155318               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106572    155319               0  2017-03-30 15:20:35  2017-02-17 00:00:00   
106573    155320               0  2017-03-26 16:22:18  2017-03-26 00:00:00   

       album.engineer  album.favorites  album.id  \
0          

In [10]:
# Retrieve query from the user
def get_user_query(df):
    print("Enter the attributes of the query as comma-separated values:")
    query_input = input()
    try:
        # Split input and convert to match DataFrame structure
        query_data = [float(x) if x.replace('.', '', 1).isdigit() else x for x in query_input.split(",")]
        query_df = pd.DataFrame([query_data], columns=df.columns)
        return query_df
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

In [11]:
# query is the song record that we want to get recommendations for 
query = get_user_query(df) 
if query is None:
    query = df.iloc[0:1]
    
distances = gower.gower_matrix(df, query)

print(distances)

Enter the attributes of the query as comma-separated values:
Error processing query: 53 columns passed, passed data had 1 columns
[[0.16981132]
 [0.2266182 ]
 [0.20788004]
 ...
 [       nan]
 [       nan]
 [       nan]]


In [12]:
# Ranking rows in the DataFrame according to their proximity to the query
distances = np.nan_to_num(distances, nan=np.inf)
distances = distances.flatten()


closest_indices = np.argsort(distances)[:10000]
print(closest_indices)

print(df.iloc[closest_indices])

[    0     2     1 ... 15463  3783 15460]
       track_id  album.comments   album.date_created  album.date_released  \
0             2               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
2             5               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
1             3               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
9           134               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
5729      10666               0  2008-11-26 01:44:45  2009-01-05 00:00:00   
...         ...             ...                  ...                  ...   
19745     31325               0  2010-06-11 09:54:02                  NaN   
15452     24793               0  2010-02-04 20:51:57  2010-02-02 00:00:00   
15463     24804               0  2010-02-04 20:51:57  2010-02-02 00:00:00   
3783       7402              -1                  NaN                  NaN   
15460     24801               0  2010-02-04 20:51:57  2010-02-02 00:00:00   

      album.engineer  album.favor